In [96]:
from ipywidgets import widgets, Layout
from IPython.display import display
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP
import binascii
import random
import math

In [97]:
#credits
#widget info help from https://ipywidgets.readthedocs.io/en/stable/
#styling help from https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Styling.html
#RSA help from https://cryptobook.nakov.com/asymmetric-key-ciphers/rsa-encrypt-decrypt-examples
#DH help from https://en.wikipedia.org/wiki/Diffie%E2%80%93Hellman_key_exchange

In [102]:
#create widgets
intro = widgets.Label(value='Anything typed while text is encoded will not load. \
                      You can encrypt your message multiple times, but you will \
                      have to decrypt in the same order.')
style = {'description_width': 'initial'}
keyIn = widgets.Text(placeholder='type here', description='Enter your plaintext:', disabled=False, style=style)
dropdown = widgets.Dropdown(options=['RSA', 'Diffe-Hellman','Elliptic curve'], value='RSA', 
                 description='Choose an encryption method:', disabled=False, style=style)
encdec = widgets.ToggleButton(description='Encrypt/Decrypt', disabled=False)
label = widgets.Label(value='Output:')
output = widgets.Output(layout=Layout(height='100px', width='900px', overflow='hidden scroll',
                                      border='1px solid black'))
#submit = widgets.Button(description='Submit', disabled=False)

#display widgets
display(intro)
display(keyIn)
display(dropdown, encdec)#, submit)
display(label)
display(output)

#create variables
plaintext = keyIn.value
encryptionMode = dropdown.value
ciphertext = plaintext
encrypted = '.'
flag = 'e'
#RSAvariables
RSAkeyPair = 0
RSApublicKey = 0
RSApublicKeyPEM = 0
RSAprivateKeyPEM = 0
#DHvariables
DHp = 23 #assume these are already agreed upon; can be changed
DHg = 5 #primitive root modulo 23
DHAsecretInt = math.floor(random.random() *10)
DHApublicKey = (DHg**DHAsecretInt)%DHp
DHBsecretInt = math.floor(random.random() *10)
DHBpublicKey = (DHg**DHBsecretInt)%DHp
DHAsecret = (DHBpublicKey**DHAsecretInt)%DHp
DHBsecret = (DHApublicKey**DHBsecretInt)%DHp
padd = chr(math.floor(random.random() *10))

#handlers & functions
def RSAkeygen():
    global RSAkeyPair, RSApublicKey, RSApublicKeyPEM, RSAprivateKeyPEM
    RSAkeyPair = RSA.generate(1024) #1024 bit key --> roughly 40 characters; standard is 3072 or 4096
    RSApublicKey = RSAkeyPair.publickey()
    RSApublicKeyPEM = RSApublicKey.exportKey()
    RSAprivateKeyPEM = RSAkeyPair.exportKey()
RSAkeygen()
    
def encr(plain, mode):
    global RSApublicKey, ciphertext, encrypted
    if (mode == 'RSA'): 
        cipher = bytes(plain, 'utf-8')
        encryptor = PKCS1_OAEP.new(RSApublicKey)
        encrypted = encryptor.encrypt(cipher)
        ciphertext = binascii.b2a_base64(encrypted).decode()
    elif (mode == 'Diffe-Hellman'):
        cipher=''
        for i in plain:
            cipher = cipher + chr(ord(i)^ord(padd));
        ciphertext = cipher
    with output:
        output.clear_output()
        print(ciphertext)

def decr(cipher, mode):
    global plaintext, encrypted
    if (mode == 'RSA'):
        decryptor = PKCS1_OAEP.new(RSAkeyPair)
        plaintext = decryptor.decrypt(encrypted).decode()
    elif (mode == 'Diffe-Hellman'):
        plain = ''
        for i in cipher:
            plain = plain + chr(ord(i)^ord(padd));
        plaintext = plain
    with output:
        output.clear_output()
        print(plaintext)

def dropdownHandler(change):
    global plaintext, ciphertext, encryptionMode, flag
    plaintext = keyIn.value
    encryptionMode = change.new
dropdown.observe(dropdownHandler, names='value')

def keyInHandler(change):
    global plaintext, ciphertext, encryptionMode, flag
    plaintext = change.new
    encryptionMode = dropdown.value
    with output:
        output.clear_output()
        print(plaintext)    
keyIn.observe(keyInHandler, names='value')

def encdecHandler(obj):
    global plaintext, ciphertext, encryptionMode, flag
    if flag == 'e':
        flag = 'd'
        encr(plaintext, encryptionMode)
    else:
        flag ='e'
        decr(ciphertext, encryptionMode)
encdec.observe(encdecHandler, 'value')

Label(value='Anything typed while text is encoded will not load.                       You can encrypt your me…

Text(value='', description='Enter your plaintext:', placeholder='type here', style=DescriptionStyle(descriptio…

Dropdown(description='Choose an encryption method:', options=('RSA', 'Diffe-Hellman', 'Elliptic curve'), style…

ToggleButton(value=False, description='Encrypt/Decrypt')

Label(value='Output:')

Output(layout=Layout(border='1px solid black', height='100px', overflow='hidden scroll', width='900px'))

In [92]:
DHp = 23 #assume these are already agreed upon; can be changed
DHg = 5 #primitive root modulo 23
DHAsecretInt = math.floor(random.random() *10)
DHApublicKey = (DHg**DHAsecretInt)%DHp
DHBsecretInt = math.floor(random.random() *10)
DHBpublicKey = (DHg**DHBsecretInt)%DHp
DHAsecret = (DHBpublicKey**DHAsecretInt)%DHp
DHBsecret = (DHApublicKey**DHBsecretInt)%DHp
padd = chr(math.floor(random.random() *10))

def encr(plain, padd):
    global ciphertext
    cipher=''
    for i in plain:
        cipher = cipher + chr(ord(i)^ord(padd));
    ciphertext = cipher
def decr(cipher, padd):
    global plaintext
    plain = ''
    for i in cipher:
        plain = plain + chr(ord(i)^ord(padd));
    plaintext = plain
    
ciphertext= ''
plaintext = 'hello world'
encrypted = ''
encr(plaintext, padd)
decr(ciphertext, padd)
print('encrypted:',ciphertext)
print('decrypted:',plaintext)

16
encrypted: lahhk$skvh`
decrypted: hello world
